# Analysising the results of the TDA experiment

How did the TDA experiment go for the initial run? For this I varied the following three parameters:

1. The number of samples considered.
2. Whether or not the labels were appended as a multiset of integer labels or as a one hot vector.
3. Random seed for the subset of the dataset considered.

Here I'll plot the mean spearman's rank between the metric order and the original order (averaged over the different seeds) against the number of samples.

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import yaml
from matplotlib.axes import Axes
from scipy.stats import spearmanr as spr

from locomoset.plots.utils import load_results

In [8]:
# load the actual scores:
with open("../configs/scores_imagenet1k.yaml", "r") as f:
    actuals = yaml.safe_load(f)

In [9]:
actuals

{'facebook/deit-tiny-patch16-224': 72.2,
 'facebook/deit-small-patch16-224': 79.9,
 'facebook/deit-base-patch16-224': 81.8,
 'google/vit-base-patch16-384': 85.49,
 'microsoft/cvt-13': 81.6}

In [13]:
# What does loading several results dictionaries look like?
results = load_results(file_paths=["../results_wp2/"])

In [22]:
len(results[0])

34

In [23]:
results[0][0]

{'save_dir': 'results_wp2',
 'models': ['facebook/deit-tiny-patch16-224',
  'facebook/deit-small-patch16-224',
  'facebook/deit-base-patch16-224',
  'microsoft/cvt-13',
  'google/vit-base-patch16-384'],
 'dataset_name': 'imagenet-1k',
 'dataset_split': 'validation',
 'n_examples': 2048,
 'random_state': 42,
 'one_hot_labels': False,
 'model_features': False,
 'pixel_square': 224,
 'result': {'results': {'facebook/deit-tiny-patch16-224': 11368.685605910974,
   'facebook/deit-small-patch16-224': 0.0,
   'facebook/deit-base-patch16-224': 0.0,
   'microsoft/cvt-13': 0.0,
   'google/vit-base-patch16-384': 0.0},
  'time': 3342.6428689956665}}

In [35]:
# test correlation:
ex_results = np.array(list(results[0][0]["result"]["results"].values()))
print(ex_results)
act_vals = np.array(list(actuals.values()))
print(act_vals)

[11368.68560591     0.             0.             0.
     0.        ]
[72.2  79.9  81.8  85.49 81.6 ]


In [41]:
print(
    f"spearmans rank correlations between \n example result and actuals: {spr(act_vals, ex_results)[0]}"
)

spearmans rank correlations between 
 example result and actuals: -0.7071067811865475


So as expected this is not well correlated so far.

In [47]:
def digest_result(result_dict, actual_vals):
    res_vals = np.array(list(result_dict["result"]["results"].values()))
    act_vals = np.array(list(actual_vals.values()))
    return {
        "n_examples": result_dict["n_examples"],
        "one_hot_labels": result_dict["one_hot_labels"],
        "correlation": spr(act_vals, res_vals)[0],
    }

In [43]:
n_examples = [128, 256, 512, 1024, 2048, 4096]

In [46]:
labels_res = {n: [] for n in n_examples}
one_hot_res = {n: [] for n in n_examples}

In [49]:
for res_dict in results[0]:
    digested_res = digest_result(res_dict, actuals)
    if digested_res["one_hot_labels"]:
        one_hot_res[digested_res["n_examples"]].append(digested_res["correlation"])
    else:
        labels_res[digested_res["n_examples"]].append(digested_res["correlation"])

/Users/edable-heath/Library/Caches/pypoetry/virtualenvs/locomoset-yXOa9fJX-py3.10/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4881: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


In [50]:
labels_res

{128: [nan, nan, nan],
 256: [nan, nan, nan],
 512: [nan, -0.7071067811865475, nan],
 1024: [-0.7071067811865475, nan, nan],
 2048: [-0.7071067811865475, -0.7071067811865475, nan],
 4096: [-0.7071067811865475, -0.7071067811865475]}

In [71]:
for res in results[0]:
    if res["n_examples"] == 1024:
        print(res)

{'save_dir': 'results_wp2', 'models': ['facebook/deit-tiny-patch16-224', 'facebook/deit-small-patch16-224', 'facebook/deit-base-patch16-224', 'microsoft/cvt-13', 'google/vit-base-patch16-384'], 'dataset_name': 'imagenet-1k', 'dataset_split': 'validation', 'n_examples': 1024, 'random_state': 42, 'one_hot_labels': True, 'model_features': False, 'pixel_square': 224, 'result': {'results': {'facebook/deit-tiny-patch16-224': 219869.40249954315, 'facebook/deit-small-patch16-224': 219938.86534636185, 'facebook/deit-base-patch16-224': 219823.8437750884, 'microsoft/cvt-13': 219805.26494509072, 'google/vit-base-patch16-384': 219871.9893949013}, 'time': 1775.7967417240143}}
{'save_dir': 'results_wp2', 'models': ['facebook/deit-tiny-patch16-224', 'facebook/deit-small-patch16-224', 'facebook/deit-base-patch16-224', 'microsoft/cvt-13', 'google/vit-base-patch16-384'], 'dataset_name': 'imagenet-1k', 'dataset_split': 'validation', 'n_examples': 1024, 'random_state': 42, 'one_hot_labels': False, 'model_f

In [73]:
one_hot_res

{128: [-0.49999999999999994, -0.7, -0.49999999999999994],
 256: [0.6, 0.7, -0.19999999999999998],
 512: [0.3, 0.3, 0.19999999999999998],
 1024: [-0.7, 0.3, -0.09999999999999999],
 2048: [0.9999999999999999, -0.49999999999999994, 0.0],
 4096: [0.09999999999999999, -0.7999999999999999]}

In [74]:
for key in one_hot_res.keys():
    print(key, np.mean(one_hot_res[key]))

128 -0.5666666666666667
256 0.36666666666666664
512 0.26666666666666666
1024 -0.16666666666666666
2048 0.16666666666666666
4096 -0.35
